In [0]:
from mlflow.tracking import MlflowClient
import mlflow

# Set the registry URI for Unity Catalog
mlflow.tracking._model_registry.utils._get_registry_uri_from_spark_session = lambda: "databricks-uc"

# Initialize the MLflow client
client = MlflowClient()

# Search for all registered models
registered_models = client.search_registered_models()

# Iterate over models in the desired namespace
for model in registered_models:
    if model.name.startswith("field_demos.ml_ops"):
        # Retrieve all versions for this model
        model_versions = client.search_model_versions(f"name='{model.name}'")
        
        if not model_versions:
            print(f"No versions found for model: {model.name}")
            continue

        best_version = None
        best_accuracy = -float('inf')
        
        # Find the version with the highest accuracy metric
        for version in model_versions:
            run = client.get_run(version.run_id)
            accuracy = run.data.metrics.get("accuracy")
            try:
                acc_value = float(accuracy) if accuracy is not None else -float('inf')
            except (TypeError, ValueError):
                acc_value = -float('inf')
            
            if acc_value > best_accuracy:
                best_accuracy = acc_value
                best_version = version
        
        if best_version is None:
            print(f"No valid accuracy found for model: {model.name}")
            continue

        print(f"Best version for model {model.name} is {best_version.version} with accuracy {best_accuracy}")
        
        # Delete all versions except for the best one
        for version in model_versions:
            if version.version != best_version.version:
                client.delete_model_version(model.name, version.version)
                print(f"Deleted model version {version.version} for model {model.name}")
        
        # Update the description of the best version to 'prod'
        client.update_model_version(model.name, best_version.version, description="prod")
        print(f"Updated model {model.name} version {best_version.version} description to 'prod'")